#### 새로운 시도를 하기 전에 한 번 지금까지 썼던 모델들 다 검토해보기로

In [55]:
# import librosa
# import librosa.display
# import IPython.display as ipd
# from IPython.display import Audio

from random import randint
import numpy as np
import pandas as pd
# import pydub #오디오 파일을 다루는 라이브러리 : 변환 조작 재생 분석 등
# import gTTS # 텍스트를 음성으로 변환하는 라이브러리. 나는 당장 쓸 일 없을 듯?

# from tqdm import tqdm
# import np_utils
# import matplotlib.pyplot as plt
# import seaborn as sns

# import winsound as sd
# import glob
# import os
# import json
# import shutil
# import sys
# import logging
# import unicodedata
# from shutil import copyfile
# import warnings
# if not sys.warnoptions:
#     warnings.simplefilter("ignore")
# warnings.filterwarnings("ignore", category=DeprecationWarning) 


from sklearn.preprocessing import LabelEncoder,StandardScaler, OneHotEncoder, scale, MinMaxScaler
# from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

import joblib
from joblib import dump, load
from xgboost import XGBClassifier 

# import keras
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping
from keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils import to_categorical

# import tensorflow
from sentence_transformers import SentenceTransformer


# from transformers import PreTrainedTokenizer




In [2]:
train_csv = pd.read_csv('e:/Data2/csv/train.csv')
valid_csv = pd.read_csv('e:/Data2/csv/valid.csv')
X = np.load('./features5.npy')
y = train_csv.iloc[:,-7:]

In [3]:
# 기본적인 사전 설정
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=5, min_lr=0.0000001) #learning rate 조절 
modelpath = './model/model_{epoch:02d}-{val_accuracy:.4f}.keras'
mcp = ModelCheckpoint(
    modelpath,     #저장할 모델의 경로
  monitor = 'val_accuracy', #val_acc를 기준으로 전보다 모델이 나아지는 걸 확인
  save_best_only = True,    #나아진 결과만 저장
#     save_weights_only=True , #이걸 써 줘야 weights.h5로 저장 가능하다.
  verbose = 1               #과정을 출력
)

#전보다 나아지지 않으면 학습중단

es = EarlyStopping(
    monitor = 'val_accuracy',
    patience = 5      # 전보다 나아지지 않아도 실행할 횟수
)

### CNN

In [42]:
def cnn_model(x_train):
    model=Sequential()
    model.add(InputLayer(shape=(x_train.shape[1],x_train.shape[2])))
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(units=7, activation='softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

    #model.summary()
    return model

In [46]:
#StandardScaler로 


scaler = StandardScaler()

   
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0,  test_size = 0.3, stratify = y)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = np.expand_dims(X_train_scaled,axis =1)
X_test_scaled = np.expand_dims(X_test_scaled,axis =1)


model = second_model(X_train_scaled)
history=model.fit(X_train_scaled, y_train, batch_size=40, epochs=50, validation_data=(X_test_scaled, y_test), callbacks=[rlrp,mcp,es])

test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print("Test Accuracy: ",test_acc)

# test 결과값 약 47.5%

Epoch 1/50
506/514 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3592 - loss: 1.6858
Epoch 1: val_accuracy did not improve from 0.50614
514/514 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.3598 - loss: 1.6847 - val_accuracy: 0.4484 - val_loss: 1.5184 - learning_rate: 0.0010
Epoch 2/50
511/514 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4484 - loss: 1.5211
Epoch 2: val_accuracy did not improve from 0.50614
514/514 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.4484 - loss: 1.5210 - val_accuracy: 0.4511 - val_loss: 1.4871 - learning_rate: 0.0010
Epoch 3/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4584 - loss: 1.4783
Epoch 3: val_accuracy did not improve from 0.50614
514/514 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.4584 - loss: 1.4783 - val_accuracy: 0.4679 - val_loss: 1.4360 - learning_rate: 0.0010
Epoch 4/50
511/514 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4747 - loss: 1.4391
Epoch 4: val_accuracy did not improve from 0.50614
514/514 ━━━━━━━━━━━━━━━━━━━━ 

In [37]:
#MinMaxScaler로 

scaler = MinMaxScaler()

   
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0,  test_size = 0.3, stratify = y)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = np.expand_dims(X_train_scaled,axis =1)
X_test_scaled = np.expand_dims(X_test_scaled,axis =1)


model = second_model(X_train_scaled)
history=model.fit(X_train_scaled, y_train, batch_size=40, epochs=50, validation_data=(X_test_scaled, y_test), callbacks=[rlrp,mcp,es])

test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print("Test Accuracy: ",test_acc)

# test 결과값 약 41.2% : CNN은 StandardScaler로

Epoch 1/50
508/514 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3723 - loss: 1.7180
Epoch 1: val_accuracy did not improve from 0.50614
514/514 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.3725 - loss: 1.7175 - val_accuracy: 0.3832 - val_loss: 1.6285 - learning_rate: 0.0010
Epoch 2/50
513/514 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3932 - loss: 1.6417
Epoch 2: val_accuracy did not improve from 0.50614
514/514 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.3932 - loss: 1.6417 - val_accuracy: 0.3830 - val_loss: 1.6234 - learning_rate: 0.0010
Epoch 3/50
513/514 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3923 - loss: 1.6229
Epoch 3: val_accuracy did not improve from 0.50614
514/514 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3923 - loss: 1.6229 - val_accuracy: 0.4051 - val_loss: 1.5886 - learning_rate: 0.0010
Epoch 4/50
506/514 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4002 - loss: 1.5998
Epoch 4: val_accuracy did not improve from 0.50614
514/514 ━━━━━━━━━━━━━━━━━━━━ 

### KNN

In [42]:
# Define the parameter grid for the random search
X_train, X_test, y_train, y_test = train_test_split(X , train_csv['감정'],random_state=0, stratify = train_csv['감정'], test_size = 0.3)



# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)



param_grid = {
    'n_neighbors': np.arange(1, 15),  # Number of neighbors
    'weights': ['uniform', 'distance'],  # Weight function
    'p': [1, 2]  # Power parameter for the Minkowski distance metric
}

# Create the KNN classifier
knn = KNeighborsClassifier()

# Perform the random search
random_search_knn = RandomizedSearchCV(
    knn, param_distributions=param_grid, n_iter=10, cv=5, random_state=42
)
random_search_knn.fit(X_train, y_train_encoded)

# Evaluate the KNN model with the best parameters on the test set
best_knn = random_search_knn.best_estimator_
y_pred_knn = best_knn.predict(X_test)
test_accuracy_knn = accuracy_score(y_test_encoded, y_pred_knn)

# Evaluate the KNN model on the training set
y_train_pred_knn = best_knn.predict(X_train)
train_accuracy_knn = accuracy_score(y_train_encoded, y_train_pred_knn)

print("Train KNN Accuracy:", train_accuracy_knn)
print("Test KNN Accuracy:", test_accuracy_knn)


#test 결과값 약 33% 나옴

Train KNN Accuracy: 0.4249378745797398
Test KNN Accuracy: 0.3358344702137335


### LSTM

In [46]:
 def lstm_model(X_train):
    model = Sequential()
    model.add(InputLayer(shape=(X_train.shape[1],X_train.shape[2])))
    model.add(LSTM(64, return_sequences=True)) 
    model.add(LSTM(32)) 
    model.add(Dense(128,activation='relu'))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify = y, test_size = 0.3)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

X_train_scaled = np.expand_dims(X_train_scaled,axis =1)
X_test_scaled = np.expand_dims(X_test_scaled,axis =1)

model = lstm_model(X_train_scaled)

h1 = model.fit(X_train_scaled,y_train,
               validation_split = 0.3,
               epochs=30,
               batch_size=30,callbacks=[rlrp,es,mcp])

#test 결과값 약 46.8% 정도

Epoch 1/30
455/479 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3301 - loss: 0.4530
Epoch 1: val_accuracy did not improve from 0.81958
479/479 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.3327 - loss: 0.4493 - val_accuracy: 0.4216 - val_loss: 0.3428 - learning_rate: 0.0010
Epoch 2/30
440/479 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4317 - loss: 0.3357
Epoch 2: val_accuracy did not improve from 0.81958
479/479 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4325 - loss: 0.3354 - val_accuracy: 0.4530 - val_loss: 0.3296 - learning_rate: 0.0010
Epoch 3/30
446/479 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4613 - loss: 0.3242
Epoch 3: val_accuracy did not improve from 0.81958
479/479 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4614 - loss: 0.3241 - val_accuracy: 0.4665 - val_loss: 0.3237 - learning_rate: 0.0010
Epoch 4/30
461/479 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4747 - loss: 0.3162
Epoch 4: val_accuracy did not improve from 0.81958
479/479 ━━━━━━━━━━━━━━━━━━━━ 

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify = y, test_size = 0.3)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

X_train_scaled = np.expand_dims(X_train_scaled,axis =1)
X_test_scaled = np.expand_dims(X_test_scaled,axis =1)

model = lstm_model(X_train_scaled)

h1 = model.fit(X_train_scaled,y_train,
               validation_split = 0.3,
               epochs=30,
               batch_size=30,callbacks=[rlrp,es,mcp])

#test 결과값 약 42.7% 정도. LSTM은 StandardScaler로

Epoch 1/30
447/479 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4046 - loss: 0.3499
Epoch 1: val_accuracy did not improve from 0.50807
479/479 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4044 - loss: 0.3498 - val_accuracy: 0.4174 - val_loss: 0.3453 - learning_rate: 0.0010
Epoch 2/30
454/479 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4147 - loss: 0.3441
Epoch 2: val_accuracy did not improve from 0.50807
479/479 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4148 - loss: 0.3440 - val_accuracy: 0.4096 - val_loss: 0.3454 - learning_rate: 0.0010
Epoch 3/30
479/479 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4193 - loss: 0.3422
Epoch 3: val_accuracy did not improve from 0.50807
479/479 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4194 - loss: 0.3421 - val_accuracy: 0.4328 - val_loss: 0.3396 - learning_rate: 0.0010
Epoch 4/30
462/479 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4339 - loss: 0.3357
Epoch 4: val_accuracy did not improve from 0.50807
479/479 ━━━━━━━━━━━━━━━━━━━━ 

### SVC

In [60]:
X_train, X_test, y_train,y_test =train_test_split(X,train_csv['감정'],random_state=5,stratify=train_csv['감정'],test_size=0.3)
#MinMAx Scaler을 이용한 특징 벡터 전처리
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


#분류기 커널 설정
clf = SVC(C=100, kernel='rbf', probability=True)
#분류기 학습
clf.fit(X_train_scaled, y_train_encoded )
#각 row의 클래스별 확률을 구하기
probabilities = clf.predict_proba(X_test_scaled)
print("첫 번째 샘플의 클래스별 확률:", probabilities[0])
#예측 결과
y_pred = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_pred, y_test_encoded)
    

    
print("정확도 :", accuracy)

#test 결과값 약 51.6% 정도

정확도 : 0.5160300136425648


In [20]:
#Scaler를 StandardScaler로

X_train, X_test, y_train,y_test =train_test_split(X,train_csv['감정'],random_state=5,stratify=train_csv['감정'],test_size=0.3)
#StandardScaler을 이용한 특징 벡터 전처리
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


#분류기 커널 설정
clf = SVC(C=100, kernel='rbf', probability=True)
#분류기 학습
clf.fit(X_train_scaled, y_train_encoded )
#각 row의 클래스별 확률을 구하기
probabilities = clf.predict_proba(X_test_scaled)
print("첫 번째 샘플의 클래스별 확률:", probabilities[0])
#예측 결과
y_pred = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_pred, y_test_encoded)
    

    
print("정확도 :", accuracy)

#test 결과값 약 47.5% 정도. SVC에서는 MinMaxScaler로

첫 번째 샘플의 클래스별 확률: [0.20003447 0.10426802 0.08509696 0.04807575 0.17558152 0.35403075
 0.03291253]
정확도 : 0.47510231923601637


### ANN

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=5,stratify=y,test_size=0.3)

scaler =StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Define the model architecture
model = Sequential()
model.add(Dense(units=256, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=7, activation='softmax'))  # Adjusted to use np.unique for flexibility

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1, validation_data=(X_test_scaled, y_test),callbacks=[rlrp,mcp,es])

# Retrieve training and validation accuracy
train_accuracy = history.history['accuracy'][-1]  # Last epoch accuracy
val_accuracy = history.history['val_accuracy'][-1]  # Last epoch validation accuracy

# Predict on test set
y_pred = model.predict(X_test_scaled)
# 임계값 설정
threshold = 0.3

# 확률이 임계값보다 크면 1로, 작으면 0으로 분류
y_pred_labels = (y_pred > threshold).astype(int)
# y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate test accuracy
test_accuracy = accuracy_score(y_test, y_pred_labels)

# Print accuracies
print('Training accuracy:', train_accuracy)
print('Validation accuracy:', val_accuracy)
print('Test accuracy:', test_accuracy)

#test 결과값 약 39.9%

Epoch 1/100


C:\Users\smhrd\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


623/642 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3924 - loss: 1.6399
Epoch 1: val_accuracy did not improve from 0.50807
642/642 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3932 - loss: 1.6378 - val_accuracy: 0.4578 - val_loss: 1.4724 - learning_rate: 0.0010
Epoch 2/100
616/642 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4463 - loss: 1.4941
Epoch 2: val_accuracy did not improve from 0.50807
642/642 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4465 - loss: 1.4937 - val_accuracy: 0.4773 - val_loss: 1.4357 - learning_rate: 0.0010
Epoch 3/100
630/642 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4760 - loss: 1.4297
Epoch 3: val_accuracy did not improve from 0.50807
642/642 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4758 - loss: 1.4300 - val_accuracy: 0.4717 - val_loss: 1.4341 - learning_rate: 0.0010
Epoch 4/100
641/642 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4723 - loss: 1.4235
Epoch 4: val_accuracy did not improve from 0.50807
642/642 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/s

In [25]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=5,stratify=y,test_size=0.3)

scaler = MinMaxScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Define the model architecture
model = Sequential()
model.add(Dense(units=256, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=7, activation='softmax'))  # Adjusted to use np.unique for flexibility

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1, validation_data=(X_test_scaled, y_test),callbacks=[rlrp,mcp,es])

# Retrieve training and validation accuracy
train_accuracy = history.history['accuracy'][-1]  # Last epoch accuracy
val_accuracy = history.history['val_accuracy'][-1]  # Last epoch validation accuracy

# Predict on test set
y_pred = model.predict(X_test_scaled)
# 임계값 설정
threshold = 0.3

# 확률이 임계값보다 크면 1로, 작으면 0으로 분류
y_pred_labels = (y_pred > threshold).astype(int)
# y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate test accuracy
test_accuracy = accuracy_score(y_test, y_pred_labels)

# Print accuracies
print('Training accuracy:', train_accuracy)
print('Validation accuracy:', val_accuracy)
print('Test accuracy:', test_accuracy)

#test 결과값 약 38.4% 큰 차이는 아니지만 StandardScaler 성능이 더 좋다.

Epoch 1/100


C:\Users\smhrd\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


640/642 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3816 - loss: 1.6601
Epoch 1: val_accuracy did not improve from 0.50807
642/642 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3816 - loss: 1.6600 - val_accuracy: 0.4048 - val_loss: 1.6065 - learning_rate: 0.0010
Epoch 2/100
629/642 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4023 - loss: 1.5928
Epoch 2: val_accuracy did not improve from 0.50807
642/642 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4023 - loss: 1.5928 - val_accuracy: 0.3906 - val_loss: 1.6358 - learning_rate: 0.0010
Epoch 3/100
625/642 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4080 - loss: 1.5766
Epoch 3: val_accuracy did not improve from 0.50807
642/642 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4081 - loss: 1.5765 - val_accuracy: 0.4173 - val_loss: 1.5588 - learning_rate: 0.0010
Epoch 4/100
626/642 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4138 - loss: 1.5649
Epoch 4: val_accuracy did not improve from 0.50807
642/642 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/s

### Xgboost

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=5,stratify=y,test_size=0.3)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05) #1000개의 가지? epoch? , 0.05 학습률
xgb.fit(X_train_scaled, y_train) #학습

y_preds = xgb.predict(X_test_scaled) #검증

print('Accuracy: %.2f' % accuracy_score(y_test,y_preds))

Accuracy: 0.29


### 확인 결과
- CNN, LSTM, SVC의 성능이 괜찮은 것으로 보임
- 이제 이걸 텍스트 모델과 결합하는 형태로 실행해 보자.
- CNN은 StandardScaler,LSTM은 StandardScaler,SVC에서는 MinMaxScaler

In [34]:
# 임베딩 함수 설정
class text_embedding():
    def __init__(self, model_name):
        self.model_name = model_name

    def fit(self, X, y=None):
        return self
  
    def transform(self, X):
        embedding_model = SentenceTransformer(self.model_name)
        embedding_vec = embedding_model.encode(train_csv['발화문'])
        X_val = np.concatenate((X, embedding_vec), axis = 1)
        return X_val

### CNN 모델(텍스트 임베딩)

In [43]:
#StandardScaler로 

pre_trained_models = ['sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens',
'sentence-transformers/multi-qa-distilbert-cos-v1',
'jhgan/ko-sroberta-multitask',
'all-distilroberta-v1',
'jhgan/ko-sbert-multitask',
'all-MiniLM-L12-v2', 'jhgan/ko-sroberta-sts']

scaler = StandardScaler()

for i in pre_trained_models:
    txt_embed = text_embedding(model_name = i)
    X_embed = txt_embed.transform(X)
   
    X_train, X_test, y_train, y_test = train_test_split(X_embed,y, random_state=0,  test_size = 0.3, stratify = y)

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_scaled = np.expand_dims(X_train_scaled,axis =1)
    X_test_scaled = np.expand_dims(X_test_scaled,axis =1)


    model = second_model(X_train_scaled)
    history=model.fit(X_train_scaled, y_train, batch_size=40, epochs=50, validation_data=(X_test_scaled, y_test), callbacks=[rlrp,mcp,es])

    
    test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
    print("Pre-trained Model: ", i)
    print("Test Accuracy: ",test_acc)
    
#결과 
#'sentence-transformers/multi-qa-distilbert-cos-v1'로 임베딩한 게 가장 좋다.(약 81.96%)
# 'jhgan/ko-sbert-multitask'과 'jhgan/ko-sroberta-sts'도 나름대로 준수
# 모델명 : model_06-0.8196.keras

C:\Users\smhrd\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch 1/50
512/514 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5671 - loss: 1.2032
Epoch 1: val_accuracy improved from 0.50807 to 0.72465, saving model to ./model/model_01-0.7246.keras
514/514 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.5676 - loss: 1.2021 - val_accuracy: 0.7246 - val_loss: 0.7837 - learning_rate: 0.0010
Epoch 2/50
512/514 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7173 - loss: 0.8205
Epoch 2: val_accuracy improved from 0.72465 to 0.73499, saving model to ./model/model_02-0.7350.keras
514/514 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.7174 - loss: 0.8203 - val_accuracy: 0.7350 - val_loss: 0.7375 - learning_rate: 0.0010
Epoch 3/50
513/514 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7594 - loss: 0.6963
Epoch 3: val_accuracy improved from 0.73499 to 0.74761, saving model to ./model/model_03-0.7476.keras
514/514 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.7594 - loss: 0.6962 - val_accuracy: 0.7476 - val_loss: 0.7252 - learning_rate: 0.0010
Epoc

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\smhrd\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\smhrd\.cache\huggingface\hub\models--sentence-transformers--multi-qa-distilbert-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Epoch 1/50
513/514 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6263 - loss: 1.0748
Epoch 1: val_accuracy did not improve from 0.81958
514/514 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.6266 - loss: 1.0738 - val_accuracy: 0.7926 - val_loss: 0.6059 - learning_rate: 0.0010
Epoch 2/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8035 - loss: 0.5994
Epoch 2: val_accuracy did not improve from 0.81958
514/514 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.8034 - loss: 0.5994 - val_accuracy: 0.7983 - val_loss: 0.5625 - learning_rate: 0.0010
Epoch 3/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8239 - loss: 0.5193
Epoch 3: val_accuracy did not improve from 0.81958
514/514 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.8239 - loss: 0.5193 - val_accuracy: 0.8032 - val_loss: 0.5662 - learning_rate: 0.0010
Epoch 4/50
513/514 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8409 - loss: 0.4671
Epoch 4: val_accuracy did not improve from 0.81958
514/514 ━━━━━━━━━━━

### LSTM 모델(텍스트 임베딩)

In [53]:
# txt_embed = text_embedding(model_name = 'sentence-transformers/multi-qa-distilbert-cos-v1') 한 번 추출했으니까 그냥
# X_embed = txt_embed.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_embed, y, random_state=0, stratify = y, test_size = 0.3)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

X_train_scaled = np.expand_dims(X_train_scaled,axis =1)
X_test_scaled = np.expand_dims(X_test_scaled,axis =1)

model = lstm_model(X_train_scaled)

history = model.fit(X_train_scaled,y_train,
               validation_split = 0.3,
               epochs=30,
               batch_size=30,callbacks=[rlrp,mcp,es])


test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print("Pre-trained Model: ", i)
print("Test Accuracy: ",test_acc)

#63퍼가 최대

Epoch 1/30
473/479 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4199 - loss: 0.4096
Epoch 1: val_accuracy did not improve from 0.81958
479/479 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4209 - loss: 0.4085 - val_accuracy: 0.5774 - val_loss: 0.2692 - learning_rate: 0.0010
Epoch 2/30
479/479 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6253 - loss: 0.2450
Epoch 2: val_accuracy did not improve from 0.81958
479/479 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6252 - loss: 0.2450 - val_accuracy: 0.6133 - val_loss: 0.2470 - learning_rate: 0.0010
Epoch 3/30
458/479 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6828 - loss: 0.2110
Epoch 3: val_accuracy did not improve from 0.81958
479/479 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6826 - loss: 0.2111 - val_accuracy: 0.6182 - val_loss: 0.2405 - learning_rate: 0.0010
Epoch 4/30
471/479 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7158 - loss: 0.1911
Epoch 4: val_accuracy did not improve from 0.81958
479/479 ━━━━━━━━━━━━━━━━━━━━ 

### SVC(텍스트 임베딩)

In [56]:
# txt_embed = text_embedding(model_name = 'sentence-transformers/multi-qa-distilbert-cos-v1') #이미 위에서 추출한 거 있으니까 그냥 그거 쓴다.
# X_embed = txt_embed.transform(X) 이것도

X_train, X_test, y_train,y_test =train_test_split(X_embed,train_csv['감정'],random_state=5,stratify=train_csv['감정'],test_size=0.3)
#MinMAx Scaler을 이용한 특징 벡터 전처리
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


#분류기 커널 설정
clf = SVC(C=100, kernel='rbf', probability=True)
#분류기 학습
clf.fit(X_train_scaled, y_train_encoded )
#각 row의 클래스별 확률을 구하기
probabilities = clf.predict_proba(X_test_scaled)
print("첫 번째 샘플의 클래스별 확률:", probabilities[0])
#예측 결과
y_pred = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_pred, y_test_encoded)
    

    
print("정확도 :", accuracy)


joblib.dump(clf, 'svm_model.pkl')

# 첫 번째 샘플의 클래스별 확률: [0.04770128 0.00425071 0.5767003  0.00182524 0.1100945  0.22532101
#  0.03410696]
# 정확도 : 0.7137335152341974
# ['svm_model.pkl']

첫 번째 샘플의 클래스별 확률: [0.04770128 0.00425071 0.5767003  0.00182524 0.1100945  0.22532101
 0.03410696]
정확도 : 0.7137335152341974


['svm_model.pkl']

## 결과 
- 위에서 나온 model_06-0.8196.keras 모델 이용해서 앞으로 작업할 것